# Ozone simulation

ATSC 595

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numba import jit

from IPython.display import clear_output

In [2]:
##################################################################
# initialize each species
initconcs = {
    "CO": 2.5e12,  # molecules / cm^3
    "HO2": 0.0,  # molecules / cm^3
    "NO": 1.25e12,  # molecules / cm^3
    "NO2": 1.25e11,  # molecules / cm^3
    "O1D": 0.0,  # molecules / cm^3
    "O3": 0.0,  # molecules / cm^3
    "OH": 0.0,  # molecules / cm^3
    # "H2O":2.5e15    # molecules / cm^3  which is 0.01% of the air at sea level
}
### assign numpy indices
# CO  -> 0
# HO2 -> 1
# NO  -> 2
# NO2 -> 3
# O1D -> 4
# O3  -> 5
# OH  -> 6

initconcs = np.array([2.5e12, 0.0, 1.25e12, 1.25e11, 0.0, 0.0, 0.0])

##################################################################
# initialize rate constants (treat H2O conc as a constant)
j01 = 1.0e-3  # (units 1/s) for {01'}  NO2 + h·nu -> NO + O3
k03 = 1.73e-15  # (units (s · molecules/cm^3)^-1 ) for {03}   O3 + NO -> (O2) + NO2
j09 = 1.0e-6  # (units 1/s) for {09}   O3 + h·nu -> (O2) + O1D
k11 = 2.14e-10  # (units (s · molecules/cm^3)^-1 ) for {11} O1D + H2O -> 2 OH
k25 = 8.54e-12  # (units (s · molecules/cm^3)^-1 ) for {25} HO2 + NO -> OH + NO2
k123 = 2.28e-14  # units (s · molecules/cm^3)^-1 for {123} OH +CO +(O2)-> HO2 + (CO2)
H2O = 2.5e15  # molecules / cm^3  which is 0.01% of the air at sea level

####################################################################
# Time step info
delt_s = 0.0001  # time step (s) for your iterations
tend_h = 2.0  # duration of forecast (hours)
tsave_m = 2.0  # how often (minutes) to save results to plot

# convert everything to seconds
tend = tend_h * 3600
tsave = tsave_m * 60
delt = delt_s

# tiny sim for testing code
#tend = 1
#tsave = 0.1
#delt = 0.0001

In [3]:
@jit
def init(c0, tend, tsave):
    """
    concs:
        initializes a dataframe with columns for each chemical species,
        sets the first row to the specified initial concentrations c0
        
    initconcs:
        creates a 1-row dataframe to pass to loop fcn, same cols as concs
    """
    time = np.arange(0, tend, tsave) # timesteps to save
    
    # create the master dataframe
    tseries = pd.DataFrame(columns=["CO", "HO2", "NO", "NO2", "O1D", "O3", "OH"], index=time)
    
    # create mini dataframe to pass into loop
    #initconcs = np.array()
    
    # fill first row with initial concentrations
    concs = c0
    tseries.loc[time[0]] = concs
    #initconcs.iloc[0] = concs
    
    return tseries, initconcs

In [4]:
@jit
def step_fwd(c_old, delt):
    """
    this function holds all of the combined prod/loss equations
    """
    c_new = c_old
    
    c_new[0] += -(k123 * c_old[6] * c_old[0]) * delt
    c_new[1] += (
        k123 * c_old[6] * c_old[0] - k25 * c_old[1] * c_old[3]
    ) * delt
    c_new[2] += (
        (j01 * c_old[3])
        - (k03 * c_old[5] * c_old[2])
        - (k25 * c_old[1] * c_old[3])
    ) * delt
    c_new[3] += (
        -(j01 * c_old[3])
        + (k03 * c_old[5] * c_old[2])
        + (k25 * c_old[1] * c_old[3])
    ) * delt
    c_new[4] += ((j09 * c_old[5]) - (k11 * c_old[4] * H2O)) * delt
    c_new[5] += (
        (j01 * c_old[3]) - (k03 * c_old[5] * c_old[2]) - (j09 * c_old[5])
    ) * delt
    c_new[6] += (
        ((2 * k11 * H2O) * c_old[4])
        + (k25 * c_old[1] * c_old[3])
        - (k123 * c_old[6] * c_old[0])
    )

    # take all negative concentrations to zero
    c_new[c_new < 0] = 0

    return c_new

In [9]:
# toy simulation
save_concs, concs = init(initconcs, tend, tsave)

In [52]:
# execute this cell as many times as you like to manually step the simulation forward
c_new = step_fwd(concs, delt)
c_new

array([2.49917565e+12, 1.64728653e+05, 1.28830646e+12, 8.66935407e+10,
       3.88679736e+00, 3.88679736e+10, 4.28083214e+06])

In [7]:
# full simulation (as spec'd by time step info in cell 1)
@jit
def do_sim(initconcs, tend, delt, tsave):
    """
    runs simulation
    """
    save_concs, concs = init(initconcs, tend, tsave)
    for T in tseries.index[1:]:
        clear_output()
        print(f"Simulating: {round(T / tend * 100, 3)}%")
        for t in np.arange(0, tsave, delt):
            concs = step_fwd(concs, delt)
        #print(f"concs at {T} seconds: {concs.values[0]}")
        save_concs.loc[T] = concs
    clear_output()
    print("Simulation complete")
    return tseries


In [53]:
tseries = do_sim(initconcs, tend, delt, tsave)
# show the result:
tseries.plot()

Simulating: 5.0%


SystemError: CPUDispatcher(<function step_fwd at 0x00000225FFAB2D30>) returned a result with an error set